### Limpieza de la base de datos
Proyecto: Análisis tutelar en temas de salud 1992 - 2019

Autor: Juan Sebastian Muñoz

Objetivo: Limpiar la información de las tutelas.

Fecha de creación: Noviembre de 2019

In [117]:
import nltk                     #Para Análisis del texto
from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import timeit                   #Para calcular tiempos de ejecución del código
import re                       #Para uso de expresiones regulares
import matplotlib.pyplot as plt #Para realizar los gráficos
import requests                 #Para hacer las solicitudes del codigo fuente de las páginas
from bs4 import BeautifulSoup   #Para transformar el código extraído a formato para poder ser usado en Python
import pandas as pd             #Decarga de pandas para uso de bases de datos
plt.rcParams["figure.figsize"] = [18.0, 8.0]

In [ ]:
Tutelas = pd.read_pickle('DB_Tutelas.pkl')
len(Tutelas)

In [118]:
pat = '>Item </strong>(.*?) <strong>\xa0\xa0\xa0Expediente </strong>\xa0\xa0(.*?)<strong>\xa0\xa0\xa0 Fecha sentencia </strong>\xa0\xa0(.*?)-(.*?)-(.*?) <strong>\xa0\xa0\xa0Sentencia </strong>\xa0\xa0<a href="(.*?)">(.*?)</a> <br/>\n<strong>Magistrado Ponente</strong>:(.*?)<br/>\n<strong>Demandante / Demandado</strong> \xa0\xa0\xa0(.*?)VS(.*?)<br/>\n</p><p align="justify"><strong>Tema:</strong>(.*?\r*\n*.*?\r*\n*.*?\r*\n*.*?\r*\n*.*?)\r*\n*\s*\s*\r*\n*\r*\n*<br/>\n<strong>Recibo Relatoria: </strong> \xa0\xa0(.*?)-(.*?)-(\d+)'
info = []
for i in range (53,55):
    html = requests.get("http://www.corteconstitucional.gov.co/relatoria/radicador/buscar.php?vs=5832&pg="+str(i)+"&ponente=&demandado=&Sentencia=&Tipo=Sentencias&busqueda=salud&conector=AND&segundotema=&anios=Todos").text
    soup = BeautifulSoup(html)
    info.append(re.findall(pat, str(soup)))

In [119]:
info_lists = [list(elem) for elem in info]
infolists = [list(a) for elem in info_lists for a in elem]
len(infolists)

199

In [120]:
info_lists = [list(elem) for elem in info]
infolists = [list(a) for elem in info_lists for a in elem]
for i in range(len(infolists)):
    infolists[i][5] = "http://www.corteconstitucional.gov.co/"+infolists[i][5]
infolists

[['5302',
  'T-4662035',
  '2015',
  '04',
  '30',
  'http://www.corteconstitucional.gov.co//relatoria/2015/T-249-15.htm',
  'T-249/15',
  'MARÍA VICTORIA CALLE CORREA        ',
  'SEGUNDO EXCELINO PINEDA SUPELANO ',
  '. RED SALUD CASANARE ESE',
  ' DERECHO A LA SALUD. NEGATIVA DE NOMBRAR MEDICO DE PLANTA. El accionante, en calidad de Personero del municipio de la Salina (Casanare) presenta la acción de tutela con el objeto de que se amparen los derechos fundamentales a la vida, a la <span style="color:black;background:#FF8000">salud</span>, a la igualdad y al saneamiento ambiental de los habitantes de dicha municipalidad, los cuales están siendo vulnerados por la entidad demandada,  por la negativa de nombrar un médico y una enfermera jefe de carácter permanente en el centro de <span style="color:black;background:#FF8000">salud</span> local. En el municipio el servicio de <span style="color:black;background:#FF8000">salud</span> se brinda sólo un par de días a la semana y a través de

In [121]:
list_tutelas=[]
for i in range(len(infolists)):
    list_tutelas.append(infolists[i])
    len(list_tutelas)

In [122]:
len(list_tutelas)

199

In [123]:
Database_tutelas = pd.DataFrame(infolists)
Database_tutelas.columns = ['Item','Expediente','Año','Mes','Día','Link', 'Sentencia', 'Magistrado_Ponente', 'Demandante', 'Demandado', 'Tema', 'Año_R', 'Mes_R', 'Día_R']

In [124]:
#Limpieza de links y formatos especiales dentro de 'tema'
def clean_color(text):
    return re.sub('<span style="color:black;background:#FF8000">.*?</span>', 'salud', text)
Database_tutelas['Tema'] = Database_tutelas['Tema'].apply(clean_color)
def clean_links(text):
    return re.sub('<a.*?a>', '', text)
Database_tutelas['Tema'] = Database_tutelas['Tema'].apply(clean_links)

In [125]:
#Deja solo al magistrado ponente:
def clean_magistrado(text):
    return re.sub('<.*?   ', '', text)
Database_tutelas['Magistrado_Ponente'] = Database_tutelas['Magistrado_Ponente'].apply(clean_magistrado)
Database_tutelas['Magistrado_Ponente'][0]

'MARÍA VICTORIA CALLE CORREA        '

In [126]:
Database_tutelas['Año'] = pd.to_numeric(Database_tutelas['Año'], errors = "coerce")
Database_tutelas['Mes'] = pd.to_numeric(Database_tutelas['Mes'], errors = "coerce")
Database_tutelas['Día'] = pd.to_numeric(Database_tutelas['Día'], errors = "coerce")
Database_tutelas['Año_R'] = pd.to_numeric(Database_tutelas['Año_R'], errors = "coerce")
Database_tutelas['Mes_R'] = pd.to_numeric(Database_tutelas['Mes_R'], errors = "coerce")
Database_tutelas['Día_R'] = pd.to_numeric(Database_tutelas['Día_R'], errors = "coerce")

In [127]:
# Añadir la columna de fecha en formato datetime

Database_tutelas['Fecha'] = pd.to_datetime(Database_tutelas.Año * 10000 + Database_tutelas.Mes * 100 + Database_tutelas.Día, format='%Y%m%d')
Database_tutelas['Fecha_relatoria'] = pd.to_datetime(Database_tutelas.Año_R * 10000 + Database_tutelas.Mes_R * 100 + Database_tutelas.Día_R, format='%Y%m%d')

In [128]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Año_R,Mes_R,Día_R,Fecha,Fecha_relatoria
0,5302,T-4662035,2015,4,30,http://www.corteconstitucional.gov.co//relator...,T-249/15,MARÍA VICTORIA CALLE CORREA,SEGUNDO EXCELINO PINEDA SUPELANO,. RED SALUD CASANARE ESE,DERECHO A LA SALUD. NEGATIVA DE NOMBRAR MEDIC...,2015,7,29,2015-04-30,2015-07-29
1,5303,T-4600934 Y OTRO ACUMULADOS,2015,5,4,http://www.corteconstitucional.gov.co//relator...,T-255/15,LUIS ERNESTO VARGAS SILVA,CARLOS EDUARDO CAICEDO PERLAZA,. CAPRECOM EPS Y OTRO,Salud. Se acumulan expedientes por unidad ...,2015,6,3,2015-05-04,2015-06-03
2,5304,T-4587990,2015,5,5,http://www.corteconstitucional.gov.co//relator...,T-256/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,CAMPO ELIAS LOPEZ MORON,. CARBONES DEL CERREJON LIMITED,DERECHO FUNDAMENTAL AL AGUA POTABLE. VULNERAC...,2016,3,1,2015-05-05,2016-03-01
3,5305,T-4492963 Y OTROS ACUMULADOS,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-274/15,JORGE IVÁN PALACIO PALACIO,MONICA VIVIAN CALDERON BARRERA Y OTRAS,. EPS COOMEVA Y OTRAS,DERECHO A LA SALUD REPRODUCTIV. TRATAMIENTO ...,2015,9,9,2015-05-12,2015-09-09
4,5306,T-4458978,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-275/15,MARÍA VICTORIA CALLE CORREA,AICARDO ANTONIO GRACIANO DAVID EN CALIDAD DE A...,. MINISTERIO DE SALUD Y PROTECCION SOCIAL Y OTROS,"El accionante, actuando en representación d...",2015,7,1,2015-05-12,2015-07-01
5,5307,T-3836925 Y OTRO ACUMULADOS,2015,5,13,http://www.corteconstitucional.gov.co//relator...,T-280/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,JAIME ERNESTO SUAREZ HERNANDEZ Y OTRO,. COLPENSIONES Y OTRO,INCLUSION EN NOMINA DE PENSIONADOS. VULNERACI...,2016,2,18,2015-05-13,2016-02-18
6,5308,D-10473,2015,5,20,http://www.corteconstitucional.gov.co//relator...,C-291/15,GLORIA STELLA ORTIZ DELGADO ...,DIEGO QUIÑONES CRUZ,. LEY 1607 DE 2012,Demanda de inconstitucionalidad contra los ...,2015,6,12,2015-05-20,2015-06-12
7,5309,T-4754352,2015,5,20,http://www.corteconstitucional.gov.co//relator...,T-294/15,GLORIA STELLA ORTIZ DELGADO,ALVARO SOLANO ARAQUE,. COLPENSIONES Y OTRO,Se atribuye la vulneración de derechos fund...,2015,6,3,2015-05-20,2015-06-03
8,5310,T-4691256,2015,5,21,http://www.corteconstitucional.gov.co//relator...,T-299/15,LUIS GUILLERMO GUERRERO PÉREZ,SOFIA,. NUEVA E.P.S. S.A.,Salud. La actora fue intervenida quirúrgi...,2015,7,7,2015-05-21,2015-07-07
9,5311,T-4713955,2015,5,22,http://www.corteconstitucional.gov.co//relator...,T-308/15,JORGE IGNACIO PRETELT CHALJUB,BEATRIZ SUAREZ CASTILLO,. SECRETARIA DE EDUCACION DEL DEPARTAMENTO DE ...,TRASLADO DE DOCENTE. PROCEDIBILIDAD POR AFECT...,2015,7,24,2015-05-22,2015-07-24


In [129]:
#Quita el doble espacio
def clean_spaces(text):
    return re.sub(' {2,}', '', text)
Database_tutelas['Magistrado_Ponente'] = Database_tutelas['Magistrado_Ponente'].apply(clean_spaces)

In [130]:
#Quita puntos, especialmente para que palabras como E.P.S. no vayan a quedar en tres palabras diferentes
def clean_dot(text):
    return re.sub('\.', '', text)
Database_tutelas['Demandado'] = Database_tutelas['Demandado'].apply(clean_dot)
def clean_comma(text):
    return re.sub(',', ' ', text)
Database_tutelas['Demandado'] = Database_tutelas['Demandado'].apply(clean_comma)

In [131]:
Database_tutelas['Demandado']

0                                 RED SALUD CASANARE ESE
1                                    CAPRECOM EPS Y OTRO
2                          CARBONES DEL CERREJON LIMITED
3                                    EPS COOMEVA Y OTRAS
4        MINISTERIO DE SALUD Y PROTECCION SOCIAL Y OTROS
5                                    COLPENSIONES Y OTRO
6                                       LEY 1607 DE 2012
7                                    COLPENSIONES Y OTRO
8                                           NUEVA EPS SA
9       SECRETARIA DE EDUCACION DEL DEPARTAMENTO DE C...
10                                  -VAPAN 500 SA Y OTRO
11                                   COOMEVA EPS Y OTROS
12                                   COOMEVA EPS Y OTROS
13                                       SALUD TOTAL EPS
14                                        UNIAPUESTAS S 
15                                             SALUDCOOP
16                                          COLPENSIONES
17      SECRETARIA DISTRITAL DE

In [132]:
#Revisar que no hayan duplicados
Database_tutelas.drop_duplicates(subset = ('Sentencia'), inplace = True)
len(Database_tutelas)

199

In [133]:
#Defino puntuación para eliminarla
puntuacion = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡¿”“•\r´–¡¿”“•\r´º'
puntuacion

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡¿”“•\r´–¡¿”“•\r´º'

In [134]:
#Defino una función que elimine la puntuación
def no_punt(string):
    transtable = string.maketrans('', '', puntuacion)
    return string.translate(transtable)
Database_tutelas['Tutela_no_punt'] = Database_tutelas['Tema'].apply(no_punt)


In [135]:
#Paso a minúsculas todas las letras
Database_tutelas['Tutela_no_punt'] = Database_tutelas['Tutela_no_punt'].apply(str.lower)

In [136]:
Database_tutelas['Demandado_no_punt'] = Database_tutelas['Demandado'].apply(no_punt)
Database_tutelas['Demandado_no_punt'] = Database_tutelas['Demandado_no_punt'].apply(str.lower)

In [137]:
for i in range(len(Database_tutelas)):
    print(Database_tutelas['Tema'][i])

 DERECHO A LA SALUD. NEGATIVA DE NOMBRAR MEDICO DE PLANTA. El accionante, en calidad de Personero del municipio de la Salina (Casanare) presenta la acción de tutela con el objeto de que se amparen los derechos fundamentales a la vida, a la salud, a la igualdad y al saneamiento ambiental de los habitantes de dicha municipalidad, los cuales están siendo vulnerados por la entidad demandada,  por la negativa de nombrar un médico y una enfermera jefe de carácter permanente en el centro de salud local. En el municipio el servicio de salud se brinda sólo un par de días a la semana y a través del profesional que presta sus servicios en la localidad vecina, pero esta atención es insuficiente si se tiene en cuenta que se trata de un lugar de difícil acceso, cuya población reclama atención continua e integral en salud.  La accionada se defiende argumentando que se trata de un municipio pequeño, que casi no factura, que prácticamente está subsidiado y que por ello no cuenta con los recursos para n

In [138]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Año_R,Mes_R,Día_R,Fecha,Fecha_relatoria,Tutela_no_punt,Demandado_no_punt
0,5302,T-4662035,2015,4,30,http://www.corteconstitucional.gov.co//relator...,T-249/15,MARÍA VICTORIA CALLE CORREA,SEGUNDO EXCELINO PINEDA SUPELANO,RED SALUD CASANARE ESE,DERECHO A LA SALUD. NEGATIVA DE NOMBRAR MEDIC...,2015,7,29,2015-04-30,2015-07-29,derecho a la salud negativa de nombrar medico...,red salud casanare ese
1,5303,T-4600934 Y OTRO ACUMULADOS,2015,5,4,http://www.corteconstitucional.gov.co//relator...,T-255/15,LUIS ERNESTO VARGAS SILVA,CARLOS EDUARDO CAICEDO PERLAZA,CAPRECOM EPS Y OTRO,Salud. Se acumulan expedientes por unidad ...,2015,6,3,2015-05-04,2015-06-03,salud se acumulan expedientes por unidad d...,caprecom eps y otro
2,5304,T-4587990,2015,5,5,http://www.corteconstitucional.gov.co//relator...,T-256/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,CAMPO ELIAS LOPEZ MORON,CARBONES DEL CERREJON LIMITED,DERECHO FUNDAMENTAL AL AGUA POTABLE. VULNERAC...,2016,3,1,2015-05-05,2016-03-01,derecho fundamental al agua potable vulneraci...,carbones del cerrejon limited
3,5305,T-4492963 Y OTROS ACUMULADOS,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-274/15,JORGE IVÁN PALACIO PALACIO,MONICA VIVIAN CALDERON BARRERA Y OTRAS,EPS COOMEVA Y OTRAS,DERECHO A LA SALUD REPRODUCTIV. TRATAMIENTO ...,2015,9,9,2015-05-12,2015-09-09,derecho a la salud reproductiv tratamiento d...,eps coomeva y otras
4,5306,T-4458978,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-275/15,MARÍA VICTORIA CALLE CORREA,AICARDO ANTONIO GRACIANO DAVID EN CALIDAD DE A...,MINISTERIO DE SALUD Y PROTECCION SOCIAL Y OTROS,"El accionante, actuando en representación d...",2015,7,1,2015-05-12,2015-07-01,el accionante actuando en representación de...,ministerio de salud y proteccion social y otros
5,5307,T-3836925 Y OTRO ACUMULADOS,2015,5,13,http://www.corteconstitucional.gov.co//relator...,T-280/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,JAIME ERNESTO SUAREZ HERNANDEZ Y OTRO,COLPENSIONES Y OTRO,INCLUSION EN NOMINA DE PENSIONADOS. VULNERACI...,2016,2,18,2015-05-13,2016-02-18,inclusion en nomina de pensionados vulneracio...,colpensiones y otro
6,5308,D-10473,2015,5,20,http://www.corteconstitucional.gov.co//relator...,C-291/15,GLORIA STELLA ORTIZ DELGADO,DIEGO QUIÑONES CRUZ,LEY 1607 DE 2012,Demanda de inconstitucionalidad contra los ...,2015,6,12,2015-05-20,2015-06-12,demanda de inconstitucionalidad contra los ...,ley 1607 de 2012
7,5309,T-4754352,2015,5,20,http://www.corteconstitucional.gov.co//relator...,T-294/15,GLORIA STELLA ORTIZ DELGADO,ALVARO SOLANO ARAQUE,COLPENSIONES Y OTRO,Se atribuye la vulneración de derechos fund...,2015,6,3,2015-05-20,2015-06-03,se atribuye la vulneración de derechos fund...,colpensiones y otro
8,5310,T-4691256,2015,5,21,http://www.corteconstitucional.gov.co//relator...,T-299/15,LUIS GUILLERMO GUERRERO PÉREZ,SOFIA,NUEVA EPS SA,Salud. La actora fue intervenida quirúrgi...,2015,7,7,2015-05-21,2015-07-07,salud la actora fue intervenida quirúrgic...,nueva eps sa
9,5311,T-4713955,2015,5,22,http://www.corteconstitucional.gov.co//relator...,T-308/15,JORGE IGNACIO PRETELT CHALJUB,BEATRIZ SUAREZ CASTILLO,SECRETARIA DE EDUCACION DEL DEPARTAMENTO DE C...,TRASLADO DE DOCENTE. PROCEDIBILIDAD POR AFECT...,2015,7,24,2015-05-22,2015-07-24,traslado de docente procedibilidad por afecta...,secretaria de educacion del departamento de c...


In [139]:
#Creo una lista de tokens
Database_tutelas['Tutela_tkn'] = Database_tutelas['Tutela_no_punt'].apply(word_tokenize)
Database_tutelas['Demandado_tkn'] = Database_tutelas['Demandado_no_punt'].apply(word_tokenize)

In [140]:
# Adquiero las stopwords en español y las elimino
stopwords_esp = stopwords.words('spanish')
def no_stopwords(tokens):
    return [p for p in tokens if p not in stopwords_esp]

In [141]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Año_R,Mes_R,Día_R,Fecha,Fecha_relatoria,Tutela_no_punt,Demandado_no_punt,Tutela_tkn,Demandado_tkn
0,5302,T-4662035,2015,4,30,http://www.corteconstitucional.gov.co//relator...,T-249/15,MARÍA VICTORIA CALLE CORREA,SEGUNDO EXCELINO PINEDA SUPELANO,RED SALUD CASANARE ESE,DERECHO A LA SALUD. NEGATIVA DE NOMBRAR MEDIC...,2015,7,29,2015-04-30,2015-07-29,derecho a la salud negativa de nombrar medico...,red salud casanare ese,"[derecho, a, la, salud, negativa, de, nombrar,...","[red, salud, casanare, ese]"
1,5303,T-4600934 Y OTRO ACUMULADOS,2015,5,4,http://www.corteconstitucional.gov.co//relator...,T-255/15,LUIS ERNESTO VARGAS SILVA,CARLOS EDUARDO CAICEDO PERLAZA,CAPRECOM EPS Y OTRO,Salud. Se acumulan expedientes por unidad ...,2015,6,3,2015-05-04,2015-06-03,salud se acumulan expedientes por unidad d...,caprecom eps y otro,"[salud, se, acumulan, expedientes, por, unidad...","[caprecom, eps, y, otro]"
2,5304,T-4587990,2015,5,5,http://www.corteconstitucional.gov.co//relator...,T-256/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,CAMPO ELIAS LOPEZ MORON,CARBONES DEL CERREJON LIMITED,DERECHO FUNDAMENTAL AL AGUA POTABLE. VULNERAC...,2016,3,1,2015-05-05,2016-03-01,derecho fundamental al agua potable vulneraci...,carbones del cerrejon limited,"[derecho, fundamental, al, agua, potable, vuln...","[carbones, del, cerrejon, limited]"
3,5305,T-4492963 Y OTROS ACUMULADOS,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-274/15,JORGE IVÁN PALACIO PALACIO,MONICA VIVIAN CALDERON BARRERA Y OTRAS,EPS COOMEVA Y OTRAS,DERECHO A LA SALUD REPRODUCTIV. TRATAMIENTO ...,2015,9,9,2015-05-12,2015-09-09,derecho a la salud reproductiv tratamiento d...,eps coomeva y otras,"[derecho, a, la, salud, reproductiv, tratamien...","[eps, coomeva, y, otras]"
4,5306,T-4458978,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-275/15,MARÍA VICTORIA CALLE CORREA,AICARDO ANTONIO GRACIANO DAVID EN CALIDAD DE A...,MINISTERIO DE SALUD Y PROTECCION SOCIAL Y OTROS,"El accionante, actuando en representación d...",2015,7,1,2015-05-12,2015-07-01,el accionante actuando en representación de...,ministerio de salud y proteccion social y otros,"[el, accionante, actuando, en, representación,...","[ministerio, de, salud, y, proteccion, social,..."
5,5307,T-3836925 Y OTRO ACUMULADOS,2015,5,13,http://www.corteconstitucional.gov.co//relator...,T-280/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,JAIME ERNESTO SUAREZ HERNANDEZ Y OTRO,COLPENSIONES Y OTRO,INCLUSION EN NOMINA DE PENSIONADOS. VULNERACI...,2016,2,18,2015-05-13,2016-02-18,inclusion en nomina de pensionados vulneracio...,colpensiones y otro,"[inclusion, en, nomina, de, pensionados, vulne...","[colpensiones, y, otro]"
6,5308,D-10473,2015,5,20,http://www.corteconstitucional.gov.co//relator...,C-291/15,GLORIA STELLA ORTIZ DELGADO,DIEGO QUIÑONES CRUZ,LEY 1607 DE 2012,Demanda de inconstitucionalidad contra los ...,2015,6,12,2015-05-20,2015-06-12,demanda de inconstitucionalidad contra los ...,ley 1607 de 2012,"[demanda, de, inconstitucionalidad, contra, lo...","[ley, 1607, de, 2012]"
7,5309,T-4754352,2015,5,20,http://www.corteconstitucional.gov.co//relator...,T-294/15,GLORIA STELLA ORTIZ DELGADO,ALVARO SOLANO ARAQUE,COLPENSIONES Y OTRO,Se atribuye la vulneración de derechos fund...,2015,6,3,2015-05-20,2015-06-03,se atribuye la vulneración de derechos fund...,colpensiones y otro,"[se, atribuye, la, vulneración, de, derechos, ...","[colpensiones, y, otro]"
8,5310,T-4691256,2015,5,21,http://www.corteconstitucional.gov.co//relator...,T-299/15,LUIS GUILLERMO GUERRERO PÉREZ,SOFIA,NUEVA EPS SA,Salud. La actora fue intervenida quirúrgi...,2015,7,7,2015-05-21,2015-07-07,salud la actora fue intervenida quirúrgic...,nueva eps sa,"[salud, la, actora, fue, intervenida, quirúrgi...","[nueva, eps, sa]"
9,5311,T-4713955,2015,5,22,http://www.corteconstitucional.gov.co//relator...,T-308/15,JORGE IGNACIO PRETELT CHALJUB,BEATRIZ SUAREZ CASTILLO,SECRETARIA DE EDUCACION DEL DEPAR

In [142]:
#Agrego una columna para la tutela y el demandado sin stopwords
Database_tutelas['Tutela_nsw'] = Database_tutelas['Tutela_tkn'].apply(no_stopwords)
Database_tutelas['Demandado_nsw'] = Database_tutelas['Demandado_tkn'].apply(no_stopwords)

In [143]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,...,Mes_R,Día_R,Fecha,Fecha_relatoria,Tutela_no_punt,Demandado_no_punt,Tutela_tkn,Demandado_tkn,Tutela_nsw,Demandado_nsw
0,5302,T-4662035,2015,4,30,http://www.corteconstitucional.gov.co//relator...,T-249/15,MARÍA VICTORIA CALLE CORREA,SEGUNDO EXCELINO PINEDA SUPELANO,RED SALUD CASANARE ESE,...,7,29,2015-04-30,2015-07-29,derecho a la salud negativa de nombrar medico...,red salud casanare ese,"[derecho, a, la, salud, negativa, de, nombrar,...","[red, salud, casanare, ese]","[derecho, salud, negativa, nombrar, medico, pl...","[red, salud, casanare]"
1,5303,T-4600934 Y OTRO ACUMULADOS,2015,5,4,http://www.corteconstitucional.gov.co//relator...,T-255/15,LUIS ERNESTO VARGAS SILVA,CARLOS EDUARDO CAICEDO PERLAZA,CAPRECOM EPS Y OTRO,...,6,3,2015-05-04,2015-06-03,salud se acumulan expedientes por unidad d...,caprecom eps y otro,"[salud, se, acumulan, expedientes, por, unidad...","[caprecom, eps, y, otro]","[salud, acumulan, expedientes, unidad, materia...","[caprecom, eps]"
2,5304,T-4587990,2015,5,5,http://www.corteconstitucional.gov.co//relator...,T-256/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,CAMPO ELIAS LOPEZ MORON,CARBONES DEL CERREJON LIMITED,...,3,1,2015-05-05,2016-03-01,derecho fundamental al agua potable vulneraci...,carbones del cerrejon limited,"[derecho, fundamental, al, agua, potable, vuln...","[carbones, del, cerrejon, limited]","[derecho, fundamental, agua, potable, vulnerac...","[carbones, cerrejon, limited]"
3,5305,T-4492963 Y OTROS ACUMULADOS,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-274/15,JORGE IVÁN PALACIO PALACIO,MONICA VIVIAN CALDERON BARRERA Y OTRAS,EPS COOMEVA Y OTRAS,...,9,9,2015-05-12,2015-09-09,derecho a la salud reproductiv tratamiento d...,eps coomeva y otras,"[derecho, a, la, salud, reproductiv, tratamien...","[eps, coomeva, y, otras]","[derecho, salud, reproductiv, tratamiento, fer...","[eps, coomeva]"
4,5306,T-4458978,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-275/15,MARÍA VICTORIA CALLE CORREA,AICARDO ANTONIO GRACIANO DAVID EN CALIDAD DE A...,MINISTERIO DE SALUD Y PROTECCION SOCIAL Y OTROS,...,7,1,2015-05-12,2015-07-01,el accionante actuando en representación de...,ministerio de salud y proteccion social y otros,"[el, accionante, actuando, en, representación,...","[ministerio, de, salud, y, proteccion, social,...","[accionante, actuando, representación, padres,...","[ministerio, salud, proteccion, social]"
5,5307,T-3836925 Y OTRO ACUMULADOS,2015,5,13,http://www.corteconstitucional.gov.co//relator...,T-280/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,JAIME ERNESTO SUAREZ HERNANDEZ Y OTRO,COLPENSIONES Y OTRO,...,2,18,2015-05-13,2016-02-18,inclusion en nomina de pensionados vulneracio...,colpensiones y otro,"[inclusion, en, nomina, de, pensionados, vulne...","[colpensiones, y, otro]","[inclusion, nomina, pensionados, vulneracion, ...",[colpensiones]
6,5308,D-10473,2015,5,20,http://www.corteconstitucional.gov.co//relator...,C-291/15,GLORIA STELLA ORTIZ DELGADO,DIEGO QUIÑONES CRUZ,LEY 1607 DE 2012,...,6,12,2015-05-20,2015-06-12,demanda de inconstitucionalidad contra los ...,ley 1607 de 2012,"[demanda, de, inconstitucionalidad, contra, lo...","[ley, 1607, de, 2012]","[demanda, inconstitucionalidad, artículos, 21,...","[ley, 1607, 2012]"
7,5309,T-4754352,2015,5,20,http://www.corteconstitucional.gov.co//relator...,T-294/15,GLORIA STELLA ORTIZ DELGADO,ALVARO SOLANO ARAQUE,COLPENSIONES Y OTRO,...,6,3,2015-05-20,2015-06-03,se atribuye la vulneración de derechos fund...,colpensiones y otro,"[se, atribuye, la, vulneración, de, derechos, ...","[colpensiones, y, otro]","[atribuye, vulneración, derechos, fundamentale...",[colpensiones]
8,5310,T-4691256,2015,5,21,http://www.corteconstitucional.gov.co//relator...,T-299/15,LUIS GUILLERMO GUERRERO PÉREZ,SOFIA,NUEVA EPS SA,...,7,7,2015-05-21,2015-07-07,salud la actora fue intervenida quirúrgic...,nueva eps sa,"[salud, la, actora, fue, intervenida, quirúrgi...","[nueva, eps

In [144]:
#Paso a texto los tokens de los demandados y de las tutelas
Database_tutelas['Tutela_text'] = Database_tutelas['Tutela_nsw'].apply(nltk.Text)
Database_tutelas['Demandado_text'] = Database_tutelas['Demandado_nsw'].apply(nltk.Text)

In [145]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,...,Fecha,Fecha_relatoria,Tutela_no_punt,Demandado_no_punt,Tutela_tkn,Demandado_tkn,Tutela_nsw,Demandado_nsw,Tutela_text,Demandado_text
0,5302,T-4662035,2015,4,30,http://www.corteconstitucional.gov.co//relator...,T-249/15,MARÍA VICTORIA CALLE CORREA,SEGUNDO EXCELINO PINEDA SUPELANO,RED SALUD CASANARE ESE,...,2015-04-30,2015-07-29,derecho a la salud negativa de nombrar medico...,red salud casanare ese,"[derecho, a, la, salud, negativa, de, nombrar,...","[red, salud, casanare, ese]","[derecho, salud, negativa, nombrar, medico, pl...","[red, salud, casanare]","(derecho, salud, negativa, nombrar, medico, pl...","(red, salud, casanare)"
1,5303,T-4600934 Y OTRO ACUMULADOS,2015,5,4,http://www.corteconstitucional.gov.co//relator...,T-255/15,LUIS ERNESTO VARGAS SILVA,CARLOS EDUARDO CAICEDO PERLAZA,CAPRECOM EPS Y OTRO,...,2015-05-04,2015-06-03,salud se acumulan expedientes por unidad d...,caprecom eps y otro,"[salud, se, acumulan, expedientes, por, unidad...","[caprecom, eps, y, otro]","[salud, acumulan, expedientes, unidad, materia...","[caprecom, eps]","(salud, acumulan, expedientes, unidad, materia...","(caprecom, eps)"
2,5304,T-4587990,2015,5,5,http://www.corteconstitucional.gov.co//relator...,T-256/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,CAMPO ELIAS LOPEZ MORON,CARBONES DEL CERREJON LIMITED,...,2015-05-05,2016-03-01,derecho fundamental al agua potable vulneraci...,carbones del cerrejon limited,"[derecho, fundamental, al, agua, potable, vuln...","[carbones, del, cerrejon, limited]","[derecho, fundamental, agua, potable, vulnerac...","[carbones, cerrejon, limited]","(derecho, fundamental, agua, potable, vulnerac...","(carbones, cerrejon, limited)"
3,5305,T-4492963 Y OTROS ACUMULADOS,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-274/15,JORGE IVÁN PALACIO PALACIO,MONICA VIVIAN CALDERON BARRERA Y OTRAS,EPS COOMEVA Y OTRAS,...,2015-05-12,2015-09-09,derecho a la salud reproductiv tratamiento d...,eps coomeva y otras,"[derecho, a, la, salud, reproductiv, tratamien...","[eps, coomeva, y, otras]","[derecho, salud, reproductiv, tratamiento, fer...","[eps, coomeva]","(derecho, salud, reproductiv, tratamiento, fer...","(eps, coomeva)"
4,5306,T-4458978,2015,5,12,http://www.corteconstitucional.gov.co//relator...,T-275/15,MARÍA VICTORIA CALLE CORREA,AICARDO ANTONIO GRACIANO DAVID EN CALIDAD DE A...,MINISTERIO DE SALUD Y PROTECCION SOCIAL Y OTROS,...,2015-05-12,2015-07-01,el accionante actuando en representación de...,ministerio de salud y proteccion social y otros,"[el, accionante, actuando, en, representación,...","[ministerio, de, salud, y, proteccion, social,...","[accionante, actuando, representación, padres,...","[ministerio, salud, proteccion, social]","(accionante, actuando, representación, padres,...","(ministerio, salud, proteccion, social)"
5,5307,T-3836925 Y OTRO ACUMULADOS,2015,5,13,http://www.corteconstitucional.gov.co//relator...,T-280/15,MARTHA VICTORIA SÁCHICA MÉNDEZ,JAIME ERNESTO SUAREZ HERNANDEZ Y OTRO,COLPENSIONES Y OTRO,...,2015-05-13,2016-02-18,inclusion en nomina de pensionados vulneracio...,colpensiones y otro,"[inclusion, en, nomina, de, pensionados, vulne...","[colpensiones, y, otro]","[inclusion, nomina, pensionados, vulneracion, ...",[colpensiones],"(inclusion, nomina, pensionados, vulneracion, ...",(colpensiones)
6,5308,D-10473,2015,5,20,http://www.corteconstitucional.gov.co//relator...,C-291/15,GLORIA STELLA ORTIZ DELGADO,DIEGO QUIÑONES CRUZ,LEY 1607 DE 2012,...,2015-05-20,2015-06-12,demanda de inconstitucionalidad contra los ...,ley 1607 de 2012,"[demanda, de, inconstitucionalidad, contra, lo...","[ley, 1607, de, 2012]","[demanda, inconstitucionalidad, artículos, 21,...","[ley, 1607, 2012]","(demanda, inconstitucionalidad, artículos, 21,...","(ley, 1607, 2012)"
7,5309,T-4754352,2015,5,20,http://www.corteconstitucional.gov.co//relator...,T-294/15,GLORIA STELLA ORTIZ DELGADO,ALVARO SOLANO ARAQUE,COLPENSIONES Y OTRO,...,2015-05-

In [146]:
Database_tutelas['Respuesta'] =[x for x in range (len(Database_tutelas))]

In [185]:
# Se intenta identificar si la tutela fue Concedida, concedida parcialmente, negada, etc.
# Negada = -1
# Concedida Parcialmente = 0.5
# Concedida, AMPARADA = 1
# Hecho superado = 0
# Improcedencia = 2
# Denegar = 3
# Tutelan los derechos = 4
# EXEQUIBILIDAD CONDICIONADA = 5
# Exequibilidad = 6
# Inhibe = 7
# DAÑO CONSUMADO = 8
# CARENCIA ACTUAL DE OBJETO = 9
# CONFIRMA PARCIALMENTE =10
# Se reitera = 11
# Inexequible = 12

for i in range(len(Database_tutelas)):
    if str(Database_tutelas['Tema'][i]).find(' NEGADA.<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGADA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('<br/>NEGADA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('<br/>NEGADA.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('\r<br/>NEGADA\r<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGARON ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGÓ ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGAR ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('NIEGA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('HECHO SUPERADO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 0
    elif str(Database_tutelas['Tema'][i]).find('CONCEDIDA PARCIALMENTE.<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 0.5
    elif str(Database_tutelas['Tema'][i]).find('CONCEDIDA.<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find(' CONCED') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('e conced') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('AMPAR') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('TUTELÓ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('Se TUTELA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('TUTELAR') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('OTORGA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('IMPROCEDENCIA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 2
    elif str(Database_tutelas['Tema'][i]).find('IMPROCEDENTE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 2
    elif str(Database_tutelas['Tema'][i]).find('DENIEG.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 3
    elif str(Database_tutelas['Tema'][i]).find('DENIEGA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 3
    elif str(Database_tutelas['Tema'][i]).find(' DENEG') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 3
    elif str(Database_tutelas['Tema'][i]).find(' TUTELAN') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 4
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBILIDAD CONDICIONADA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 5
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBLE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 6
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBLE.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 6
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBILIDAD') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 6
    elif str(Database_tutelas['Tema'][i]).find('INHIBE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 7
    elif str(Database_tutelas['Tema'][i]).find('INHIBIDA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 7
    elif str(Database_tutelas['Tema'][i]).find('INHIBID.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 7
    elif str(Database_tutelas['Tema'][i]).find('DAÑO CONSUMADO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 8
    elif str(Database_tutelas['Tema'][i]).find('CARENCIA ACTUAL DE OBJETO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 9
    elif str(Database_tutelas['Tema'][i]).find('CONFIRMA PARCIALMENTE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 10
    elif str(Database_tutelas['Tema'][i]).find('Se reitera') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 11
    elif str(Database_tutelas['Tema'][i]).find('INEXEQUIBLE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 12
    elif str(Database_tutelas['Tema'][i]).find('ESTARSE A LO RESUELTO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 13
    else:
        Database_tutelas.loc[[i], 'Respuesta'] = 'NA'

In [186]:
Database_tutelas.loc[54]

Item                                                               5356
Expediente                                                    T-4908207
Año                                                                2015
Mes                                                                   8
Día                                                                   5
Link                  http://www.corteconstitucional.gov.co//relator...
Sentencia                                                      T-489/15
Magistrado_Ponente                           JORGE IVÁN PALACIO PALACIO
Demandante                                    DANIEL PALACIOS MURILLO  
Demandado              ADMINISTRACION TEMPORAL DE LA EDUCACION DEL C...
Tema                   IMPROCEDENCIA DE TRASLADO DE DOCENTE NACIONAL...
Año_R                                                              2015
Mes_R                                                                 8
Día_R                                                           

In [187]:
for i in range (len(Database_tutelas)):
    if Database_tutelas['Respuesta'][i] == 'NA':
        print(Database_tutelas['Item'][i] + '-' + Database_tutelas['Respuesta'][i])

5322-NA
5323-NA
5333-NA
5334-NA
5337-NA
5340-NA
5341-NA
5345-NA
5350-NA
5351-NA
5352-NA
5353-NA
5354-NA
5355-NA
5357-NA
5358-NA
5360-NA
5400-NA
5405-NA
5434-NA
5442-NA
5463-NA
5464-NA
5467-NA
5487-NA
5500-NA


In [188]:
Database_tutelas.to_pickle('DbTutelas.pkl')